# Pandas `merge_asof()` ile Yakın Değerlere Göre Birleştirme

Bu rehber, Pandas'ın `merge_asof()` fonksiyonunu kullanarak sıralı verileri en yakın anahtara göre nasıl birleştireceğimizi anlatır. Standart `merge` fonksiyonu tam eşleşme ararken, `merge_asof()` belirli bir tolerans dahilinde geriye, ileriye veya en yakın değere göre birleştirme yapmamızı sağlar.

Bu, özellikle farklı zamanlarda kaydedilmiş verileri (örneğin, hisse senedi alım-satım işlemleri ve o andaki fiyat teklifleri) eşleştirmek için çok kullanışlıdır.

### `merge_asof()` Kullanımının Temel Kuralları
- Birleştirme yapılacak iki DataFrame de **birleştirme anahtarına (`on` parametresi) göre sıralanmış olmalıdır.** Genellikle bu bir zaman sütunudur.
- `by` parametresi ile belirtilen sütun(lar)da tam eşleşme aranır (SQL'deki gibi gruplama).
- `on` parametresi ile belirtilen sütunda ise yakın eşleşme aranır. Bu sütun sayısal (int, float) veya datetime türünde olmalıdır.

In [ ]:
import pandas as pd

#### Örnek Veri Seti Oluşturalım
Bir hisse senedi için belirli zamanlarda yapılan alım-satım (`trades`) ve anlık fiyat tekliflerini (`quotes`) içeren iki DataFrame oluşturalım.

In [ ]:
# Anlık fiyat teklifleri (quotes) - sık sık güncellenir
quotes = pd.DataFrame({
    'time': pd.to_datetime([
        '2025-07-20 10:00:00',
        '2025-07-20 10:00:01',
        '2025-07-20 10:00:04',
        '2025-07-20 10:00:07',
        '2025-07-20 10:00:12',
        '2025-07-20 10:00:15'
    ]),
    'ticker': ['MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT', 'MSFT'],
    'bid': [150.01, 150.02, 150.00, 150.03, 150.04, 150.02],
    'ask': [150.03, 150.04, 150.02, 150.05, 150.06, 150.03]
})

# Yapılan alım-satım işlemleri (trades) - belirli anlarda gerçekleşir
trades = pd.DataFrame({
    'time': pd.to_datetime([
        '2025-07-20 10:00:02.100',
        '2025-07-20 10:00:08.500',
        '2025-07-20 10:00:13.200'
    ]),
    'ticker': ['MSFT', 'MSFT', 'MSFT'],
    'price': [150.03, 150.04, 150.05],
    'quantity': [50, 75, 60]
})

print("--- Fiyat Teklifleri (Quotes) ---")
print(quotes)
print("
--- Alım-Satım İşlemleri (Trades) ---")
print(trades)

### 1. Varsayılan Davranış: Geriye Doğru Eşleştirme (`direction='backward'`)
`merge_asof()` varsayılan olarak, sol DataFrame'deki (`trades`) her bir satır için, sağ DataFrame'de (`quotes`) o satırın zamanından (`time`) önceki **en son** kaydı bulur.

In [ ]:
# merge_asof işlemi (varsayılan yön 'backward')
# `on` ile yakın eşleşme, `by` ile tam eşleşme yapılır.
merged_df = pd.merge_asof(trades, quotes, on='time', by='ticker')

print("--- Geriye Doğru Eşleştirme Sonucu ---")
print(merged_df)

**Açıklama:**
- **İlk işlem (10:00:02.100):** Bu zamandan önceki en son teklif `10:00:01`'deki fiyattır. Sonuçta bu satırın bilgileri (bid: 150.02, ask: 150.04) eklenir.
- **İkinci işlem (10:00:08.500):** Bu zamandan önceki en son teklif `10:00:07`'deki fiyattır.
- **Üçüncü işlem (10:00:13.200):** Bu zamandan önceki en son teklif `10:00:12`'deki fiyattır.

### 2. Tolerans Belirleme (`tolerance` parametresi)
Eşleşmenin sadece belirli bir zaman aralığı içinde olmasını istiyorsak `tolerance` parametresini kullanırız. Bu aralık dışındaki eşleşmeler `NaN` olarak döner.

In [ ]:
# Sadece 2 saniye tolerans ile eşleştirme
merged_with_tolerance = pd.merge_asof(
    trades, 
    quotes, 
    on='time', 
    by='ticker', 
    tolerance=pd.Timedelta('2s')
)

print("--- 2 Saniye Tolerans ile Eşleştirme ---")
print(merged_with_tolerance)

**Açıklama:**
Örneğimizde, tüm eşleşmeler 2 saniye içinde olduğu için sonuç değişmedi. Eğer bir işlem, kendisinden önceki en yakın tekliften 2 saniyeden daha geç olsaydı, o satır için `bid` ve `ask` değerleri `NaN` olacaktı.

### 3. Yön Belirleme (`direction` parametresi)
Eşleştirmenin yönünü değiştirebiliriz:
- `forward`: İleriye dönük en yakın kaydı bulur.
- `nearest`: Her iki yöndeki (ileri veya geri) mutlak olarak en yakın kaydı bulur.

In [ ]:
# En yakın kaydı bulma (direction='nearest')
merged_nearest = pd.merge_asof(
    trades, 
    quotes, 
    on='time', 
    by='ticker', 
    direction='nearest'
)

print("--- En Yakın Değere Eşleştirme (direction='nearest') ---")
print(merged_nearest)

**Açıklama:**
- **İlk işlem (10:00:02.100):** Bu zamana en yakın teklifler `10:00:01` (1.1s fark) ve `10:00:04` (1.9s fark). En yakını `10:00:01` olduğu için o seçilir.
- **İkinci işlem (10:00:08.500):** En yakın teklifler `10:00:07` (1.5s fark) ve `10:00:12` (3.5s fark). En yakını `10:00:07`'dir.
- **Üçüncü işlem (10:00:13.200):** En yakın teklifler `10:00:12` (1.2s fark) ve `10:00:15` (1.8s fark). En yakını `10:00:12`'dir.

### Özet
`pd.merge_asof()`, özellikle zaman damgalı veya sıralı verilerle çalışırken, tam eşleşme olmayan durumlar için güçlü ve esnek bir araçtır. `direction` ve `tolerance` gibi parametrelerle birleştirme mantığını ihtiyaçlarınıza göre kolayca ayarlayabilirsiniz.